In [1]:


from SBLMDCOVDOCK.SBLSettings import DOCKING_Settings
from SBLMDCOVDOCK.md_setuptools import plot_xvg
import pickle
from copy import deepcopy
import subprocess

import pandas as pd
import os
import glob

# import gmxapi as gmx
import subprocess
import matplotlib.pyplot as plt

settings = DOCKING_Settings()
main_dir = os.getcwd()
import shutil

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"],
                           "Cat_Ser": [64, 70, 67],
                           "start": [4, 23, 21]})
                           

dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"


In [25]:
#2. Generate PDBQT files for protein
# break
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
print(dir)
for pdb in structures.PDBID:
    break
    script_name = 'prepare_receptor4.py'
    prepare_rec_path = os.path.join("..","..",os.path.join(settings.config, settings.parent, script_name))
    pdb_file =  "APO_" +pdb + "_npt" + ".pdb"
    # pdb_file =  "APO_" +pdb + "_chimeraFH" + ".pdb"

    pdb_path = os.path.join(dir, pdb_file)
    cat_ser = str(structures[structures.PDBID == pdb].Cat_Ser.values[0])
    start = str(structures[structures.PDBID == pdb].start.values[0])
    out_pdb_name = "APO_" +pdb + "_ADT" + ".pdb" # extract from MD

    out_pdb_path = os.path.join(dir, out_pdb_name)
    # renumber atoms

    # !pdb_reatom -1 $pdb_path > $out_pdb_path

    !pdb_reres -$start $pdb_path | pdb_chain -A > $out_pdb_path

    # repair_



    pdbqt_name = out_pdb_name.replace(".pdb", ".pdbqt")


    prepare_receptor_command = [pythonsh, prepare_rec_path,
                                "-r", out_pdb_path,
                                "-o", pdbqt_name,
                                "-A", "bonds_hydrogens",
                                "-U", "nphs_lps",
                                "-e", 
                                "-v",

                                ]

    print(" ".join(prepare_receptor_command))
    subprocess.run(prepare_receptor_command, cwd = dir,check=True)

    # script_name = 'repair_ligand4.py'
    # script_path = os.path.join("..","..",os.path.join(settings.config, settings.parent, script_name))
    # repair_pdbqt_command = [
    #     pythonsh,
    #     script_path,
    #     "-f", pdbqt_name,
    #     "-v",
    # ]
    # print(" ".join(repair_pdbqt_command))
    # # try:
    # subprocess.run(repair_pdbqt_command, cwd = dir,check=True)
    # except:
    #     pass
    break

/home/alexi/Documents/SBL_MD_CovDock/topology/Dock


/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh ../../config/Dock/prepare_receptor4.py -r /home/alexi/Documents/SBL_MD_CovDock/topology/Dock/APO_6T3D_ADT.pdb -o APO_6T3D_ADT.pdbqt -A bonds_hydrogens -U nphs_lps -e -v
set verbose to  True
read  /home/alexi/Documents/SBL_MD_CovDock/topology/Dock/APO_6T3D_ADT.pdb
setting up RPO with mode= automatic and outputfilename=  APO_6T3D_ADT.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= True
adding gasteiger charges to peptide
'Deleting non-standard residues: from APO_6T3D_ADT


In [26]:
import os
import subprocess

sdf_complex_dir = os.path.join(settings.structures_output, "Ligand_Complex")
for idx, row in structures.iterrows():
    break
    pdb = row["PDBID"]
    cat_ser = row["Cat_Ser"]
    pdb_file =  "APO_" +pdb + "_ADT" + ".pdb" # extract from MD
    pdb_file = os.path.join(dir, pdb_file)
    ser_pdb =  "APO_" +pdb + "_ADT" + "_SER"+".pdb" # extract from MD
    ser_out = os.path.join(dir, ser_pdb)
    print(pdb_file, ser_pdb)

    # extract the serine from the receptor using subprocess

    !pdb_selres -$cat_ser $pdb_file | pdb_reatom | pdb_tidy > $ser_out

    
    sdf_out = ser_pdb.replace(".pdb", ".sdf")
    # convert the serine to SDF and protonate it using subprocess
    subprocess.run(["obabel", ser_pdb, "-O", sdf_out], cwd=dir,check=True)


/home/alexi/Documents/SBL_MD_CovDock/topology/Dock/APO_6T3D_ADT.pdb APO_6T3D_ADT_SER.pdb
/home/alexi/Documents/SBL_MD_CovDock/topology/Dock/APO_5UL8_ADT.pdb APO_5UL8_ADT_SER.pdb


1 molecule converted


/home/alexi/Documents/SBL_MD_CovDock/topology/Dock/APO_1K55_ADT.pdb APO_1K55_ADT_SER.pdb


0 molecules converted
1 molecule converted


In [31]:
sdf_complex_dir = os.path.join(settings.structures_output, "Ligand_Complex")
sdf_complex_out = os.path.join(settings.structures_output, "Complex_Aligned")
# break
os.makedirs(sdf_complex_dir, exist_ok=True)
os.makedirs(sdf_complex_out, exist_ok=True)
# perform mcs alignment for each pdb and ligand
for pdb in structures.PDBID:
    pdb = "6T3D"
    # break
    for lig in os.listdir(sdf_complex_dir):
        print(lig)
        lig_sdf = lig 
        lig = lig.replace(".sdf", "").split("_")[0]

        sdf_out = f"{pdb}_SER_{lig}.sdf"
        template_ser_sdf = "APO_" +pdb + "_ADT" + "_SER"+".sdf" # extract from MD
        template_ser_sdf = os.path.join(settings.topology, settings.parent, template_ser_sdf)  

        lig_sdf = os.path.join(sdf_complex_dir, lig_sdf)
        sdf_out = os.path.join(sdf_complex_out, sdf_out)

        script_name = "mcs_constrained_alignment_SER.py"
        mcs_script_path = os.path.join(settings.config, settings.parent, script_name)
        mcs_command = [mcs_script_path,
                       template_ser_sdf,
                       lig_sdf,
                       sdf_out,
                       str(1), str(2), str(1), # default is 100???
                       str(0)]
        
        print(" ".join(mcs_command))
        subprocess.run(mcs_command)
        # break
    break

PBC_6UQU.sdf
config/Dock/mcs_constrained_alignment_SER.py topology/Dock/APO_6T3D_ADT_SER.sdf prod_structures/Ligand_Substrate/Ligand_Complex/PBC_6UQU.sdf prod_structures/Ligand_Substrate/Complex_Aligned/6T3D_SER_PBC.sdf 1 2 1 0
Error for molecule at position 0


/home/alexi/miniconda3/envs/cudatoolkit_martini/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[15:21:23] Explicit valence for atom # 6 B, 4, is greater than permitted
[15:21:23] ERROR: Could not sanitize molecule ending on line 35
[15:21:23] ERROR: Explicit valence for atom # 6 B, 4, is greater than permitted


CXB_1NYM.sdf
config/Dock/mcs_constrained_alignment_SER.py topology/Dock/APO_6T3D_ADT_SER.sdf prod_structures/Ligand_Substrate/Ligand_Complex/CXB_1NYM.sdf prod_structures/Ligand_Substrate/Complex_Aligned/6T3D_SER_CXB.sdf 1 2 1 0
Error for molecule at position 0


/home/alexi/miniconda3/envs/cudatoolkit_martini/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[15:21:23] Explicit valence for atom # 14 B, 4, is greater than permitted
[15:21:23] ERROR: Could not sanitize molecule ending on line 51
[15:21:23] ERROR: Explicit valence for atom # 14 B, 4, is greater than permitted


BX9_6JN4.sdf
config/Dock/mcs_constrained_alignment_SER.py topology/Dock/APO_6T3D_ADT_SER.sdf prod_structures/Ligand_Substrate/Ligand_Complex/BX9_6JN4.sdf prod_structures/Ligand_Substrate/Complex_Aligned/6T3D_SER_BX9.sdf 1 2 1 0
Error for molecule at position 0


...

/home/alexi/miniconda3/envs/cudatoolkit_martini/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[15:22:08] Explicit valence for atom # 15 B, 4, is greater than permitted
[15:22:08] ERROR: Could not sanitize molecule ending on line 53
[15:22:08] ERROR: Explicit valence for atom # 15 B, 4, is greater than permitted


Error for molecule at position 0
0NB_4E3L.sdf
config/Dock/mcs_constrained_alignment_SER.py topology/Dock/APO_6T3D_ADT_SER.sdf prod_structures/Ligand_Substrate/Ligand_Complex/0NB_4E3L.sdf prod_structures/Ligand_Substrate/Complex_Aligned/6T3D_SER_0NB.sdf 1 2 1 0


/home/alexi/miniconda3/envs/cudatoolkit_martini/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[15:22:08] Explicit valence for atom # 6 B, 4, is greater than permitted
[15:22:08] ERROR: Could not sanitize molecule ending on line 58
[15:22:08] ERROR: Explicit valence for atom # 6 B, 4, is greater than permitted


Error for molecule at position 0
BJP_1ERO.sdf
config/Dock/mcs_constrained_alignment_SER.py topology/Dock/APO_6T3D_ADT_SER.sdf prod_structures/Ligand_Substrate/Ligand_Complex/BJP_1ERO.sdf prod_structures/Ligand_Substrate/Complex_Aligned/6T3D_SER_BJP.sdf 1 2 1 0
Error for molecule at position 0


/home/alexi/miniconda3/envs/cudatoolkit_martini/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[15:22:09] Explicit valence for atom # 11 B, 4, is greater than permitted
[15:22:09] ERROR: Could not sanitize molecule ending on line 92
[15:22:09] ERROR: Explicit valence for atom # 11 B, 4, is greater than permitted


In [6]:
# convert comlplex SDF to PDB 
sdf_complex_dir = os.path.join(settings.structures_output, "Ligand_Complex")
sdf_complex_out = os.path.join(settings.structures_output, "Complex_Aligned")
pdb_complex_out = os.path.join(settings.structures_output, "Aligned_PDB")
break
os.makedirs(pdb_complex_out, exist_ok=True)
# Convert sdf to pdb
# for pdb in structures.PDBID:

for lig in os.listdir(sdf_complex_out):

    pbd_name = lig.replace(".sdf", ".pdb")
    pdb_path = os.path.join("..","Aligned_PDB", pbd_name)


    obabel_command = [
        "obabel",
        lig,
        "-O",pdb_path,
        "-l", "1",
    ]

    print(" ".join(obabel_command))
    subprocess.run(obabel_command, cwd=sdf_complex_out)
        
        # 


SyntaxError: 'break' outside loop (2988355548.py, line 5)

In [7]:
import os

# Assuming you have a DataFrame named "structures" containing the necessary data
# and "pdbqt_complex_out" is the directory containing PDB files

pdb = "6T3D"
resn = "SER"

# Assuming structures DataFrame has a column named "PDBID" and "Cat_Ser"
cat_ser = structures[structures.PDBID == pdb].Cat_Ser.values[0]

for lig in os.listdir(pdb_complex_out):
    break
    if lig.endswith('.pdb'):  # Make sure to process only PDB files
        print(lig)
        pdb_path = os.path.join(pdb_complex_out, lig)

        with open(pdb_path, "r+") as f:
            lines = f.readlines()

            new_lines = []
            new_residue_index = cat_ser  # Starting index for the modified residue numbers

            for line in lines:
                # print(line)
                if line.startswith("ATOM") or line.startswith("HETATM"):
                    atom_serial = int(line[6:11])
                    residue_index = int(line[22:26])
                    old_resname = line[17:20]
                    print(atom_serial, residue_index, old_resname)
                    print(line)
                    # Update residue name and residue number
                    if old_resname != resn and residue_index != cat_ser:
                        line = line[:17] + resn + line[20:]
                        line = line[:22] + str(new_residue_index).rjust(4) + line[26:]
                    print(line)   
                    new_lines.append(line)

            # Write modified lines back to the file
            f.seek(0)
            f.writelines(new_lines)
            f.truncate()
        # break

print("Residue renaming and numbering changes completed.")


Residue renaming and numbering changes completed.


In [8]:
#2. Generate PDBQT files for the ligands
pdbqt_complex_out = os.path.join(settings.structures_output, "Aligned_PDBQT")
# break
os.makedirs(pdbqt_complex_out, exist_ok=True)

script_name = 'prepare_receptor4.py'
prepare_rec_path = os.path.join(os.path.join(settings.config, settings.parent, script_name))

for lig in os.listdir(pdb_complex_out):


    pdb_path = os.path.join(pdb_complex_out, lig)
    pdbqt_path = os.path.join(pdbqt_complex_out, lig.replace(".pdb", ".pdbqt"))

    prepapre_cov_lig_command = [pythonsh, prepare_rec_path,
                                "-r", pdb_path,
                                "-o", pdbqt_path,
                                "-A", "bonds_hydrogens",
                                "-v"]
    
    print(" ".join(prepapre_cov_lig_command))
    subprocess.run(prepapre_cov_lig_command)

    # remove empty files
    # if os.stat(pdbqt_path).st_size == 0:
    #     os.remove(pdbqt_path)
    #     print("Removed empty file", pdbqt_path)




SyntaxError: 'break' outside loop (2270877391.py, line 3)

In [9]:
#3. Generate flexible/rigid PDBQT files for both the ligands and the receptors
flex_out = os.path.join(settings.structures_output, "PDBQT_Flex")
os.makedirs(flex_out, exist_ok=True)
# 3.1 Generate flex and rigid PDBQT files for the ligands
script_name = 'prepare_flexreceptor4.py'

prepare_flex_path = os.path.join(os.path.join(settings.config, settings.parent, script_name))


pdb = "6T3D"
for lig in os.listdir(pdbqt_complex_out):
    break
    cat_ser = structures[structures.PDBID == pdb].Cat_Ser.values[0]


    lig_path = os.path.join(pdbqt_complex_out, lig)

    flex_out_path = os.path.join(flex_out, lig)

    prepare_flex_lig_command = [pythonsh, prepare_flex_path,
                                "-r", lig_path,
                                "-s", "SER"+str(cat_ser),
                                "-g", flex_out_path.replace(".pdbqt", "_rigid.pdbqt"),
                                "-x", flex_out_path.replace(".pdbqt", "_flex.pdbqt"),
                                "-v"]


    print(" ".join(prepare_flex_lig_command))
    try:
        subprocess.run(prepare_flex_lig_command, check=True)
    except:
        print("Error in", lig)
        pass



In [10]:

# break
# 3.2 Generate flex and rigid PDBQT files for the receptors

for pdb in structures.PDBID:
    break
    pdb = "6T3D"
    cat_ser = structures[structures.PDBID == pdb].Cat_Ser.values[0]

    # pdb_file = f"APO_{pdb}_chimeraFH.pdb"

    out_pdb_name = "APO_" +pdb + "_ADT" + ".pdb"
    pdbqt_name = out_pdb_name.replace(".pdb", ".pdbqt")

    pdbqt_path = os.path.join(dir, pdbqt_name)

    prepare_flex_rec_command = [pythonsh, prepare_flex_path,
                                "-r", pdbqt_path,
                                "-s", "SER"+str(cat_ser),
                                "-g", pdbqt_path.replace(".pdbqt", "_rigid.pdbqt"),
                                "-x", pdbqt_path.replace(".pdbqt", "_flex.pdbqt"),
                                "-v"]

    print(" ".join(prepare_flex_rec_command))
    subprocess.run(prepare_flex_rec_command,check=True)
    break

In [11]:
for pdb in structures.PDBID:
    break
    # pdb_name = "APO_" +pdb + "_ADT_flex" + ".pdbqt"

    # orig_path = os.path.join(dir, pdb_name)

    # dest_path = os.path.join(flex_out, pdb_name)
    # print(os.getcwd())
    # # copy files to the flex_out directory
    # shutil.copy(orig_path, dest_path)
    # print(orig_path, dest_path)
    pdb_name = "APO_" +pdb + "_ADT_rigid" + ".pdbqt"
    orig_path = os.path.join(dir, pdb_name)

    dest_path = os.path.join(flex_out, pdb_name)
    shutil.copy(orig_path, dest_path)
    print(orig_path, dest_path)


In [12]:
pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"
print(flex_out)


for pdb in structures.PDBID:

    for lig in os.listdir(flex_out):
        if "_flex.pdbqt" not in lig:
            continue
        # break
        print(pdb, lig)


        pdb = "6T3D"
        
        break
        print(os.getcwd())
        script_name = "prepare_gpf4.py"
        prepare_grid_path = os.path.join("..","..","..",os.path.join(settings.config, settings.parent, script_name))

        pdb_name = "APO_" +pdb + "_ADT_rigid" + ".pdb"
        pdbqt_name = pdb_name.replace(".pdb", ".pdbqt") #.replace("_flex", "_rigid")
        print(pdbqt_name)
        ligand_name = lig
        
        lig = lig.replace(".pdbqt", "")



        prep_grid_command = [pythonsh,
                            prepare_grid_path,
                            "-r", pdbqt_name,
                            "-l", ligand_name,
                            "-x", ligand_name,
                            "-o", lig + ".gpf", 
                            # "-p", "ligand_types='H,HD,HS,C,A,N,NA,NS,OA,OS,F,Mg,Mn,Zn,Ca,Fe,Cl,Br,I,SA,S,Se,P,GA,GC,GN,GO,GP,G'",
                            
                            "-v"]
        
        # print(" ".join(prep_grid_command))
        dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"
        try:
            subprocess.run(prep_grid_command, cwd=flex_out)

            
        except:
            pass
    # break
    # break
# pythonsh= "/home/alexi/Documents/ADFRsuite_x86_64Linux_1.0/bin/pythonsh"



        subprocess.run(["touch", "empty"], cwd=flex_out, check=True)

# for lig in os.listdir(flex_out):
#     for pdb in structures.PDBID:
        pdb = "6T3D"
        print(os.getcwd())
        script_name = "prepare_dpf4.py"
        prepare_dock_path = os.path.join("..","..","..",os.path.join(settings.config, settings.parent, script_name))

        prep_dock_command = [pythonsh,
                            prepare_dock_path,
                            "-l", ligand_name,
                            "-x", ligand_name,
                            "-r", pdbqt_name,
                            "-o", lig+ ".dpf" ,
                            "-p", "move='empty'",
                            # "-p", "rmsdatoms='all'"
                            "-v"]

        # print(" ".join(prep_dock_command))
        dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"

        # subprocess.run(prep_dock_command, cwd=flex_out, check=True)

        try:
            print(" ".join(prep_dock_command))
            subprocess.run(prep_dock_command, cwd=flex_out, check=True)
            
            with open(os.path.join(flex_out, lig + ".dpf"), "r+") as f:
                lines = f.readlines()

                new_lines = []
                for line in lines:
                    if line.startswith("unbound_model"):
                        line = "unbound_energy 0.0      # COVALENT " + "\n"
                    new_lines.append(line)

                new_lines.append("rmsdatoms all     # COVALENT "+ "\n")

                # Write modified lines back to the file
                f.seek(0)
                f.writelines(new_lines)
                f.truncate()
        except:
            pass

    break


prod_structures/Ligand_Substrate/PDBQT_Flex
6T3D 6T3D_SER_1S6_flex.pdbqt


In [13]:
# 5. Run AutoGrid and AutoDock



autodockgpu_dir = "/home/alexi/Documents/AutoDock-GPU/bin"
engine = "CUDA_64"

autodockgpu = f"autodock_{engine}wi"
auto_dock = os.path.join(autodockgpu_dir, autodockgpu)
docked = []
# for pdb in structures.PDBID:
pdb="6T3D"
for lig in os.listdir(flex_out):
    if "_flex.dpf" not in lig:
        continue
    lig = lig.replace(".dpf", "")
    print(lig)

# for pdb in structures.PDBID:
    # print("Docking ", lig, " to ", pdb)
    name = "APO_" +pdb + "_ADT"+ "_rigid"
    pdb_name =  name + ".pdb"
    pdbqt_name = pdb_name.replace(".pdb", ".pdbqt")

    grid_name = lig +".gpf"
    grid_log = grid_name.replace(".gpf", ".glg")
    grid_log = f"{lig}_{name}.glg"


    autogrid_command = ["autogrid4", 
                        "-p", grid_name, 
                        "-l", grid_log, 
                        # "-d", f"{lig}_{name}.maps.fld",
                        # "-o", f"{lig}_{name}.maps.map",
    ]
                        # "-v"]

    dock_name = lig + ".dpf"
    dock_log = dock_name.replace(".dpf", ".dlg")

    autodock_command = [auto_dock, 
                        # "-p", dock_name, 
                        # "-l", dock_log, 
                        "-M",  f"{name}.maps.fld", 
                        "-L", f"{lig}.pdbqt",
                        "--lsmet", "ad",
                        # "--psize", "50",
                        # "--nrun", "50",
                        "-v"]


    dir = "/home/alexi/Documents/SBL_MD_CovDock/topology/Dock"
    # print(os.listdir(dir))
    try:
        subprocess.run(autogrid_command, check=True,cwd=flex_out)
        # subprocess.run(autodock_command, check=True,cwd=flex_out)        
        # check that the map fld contains ligand - otherwise command does not run before file is printed
        with open(os.path.join(flex_out, f"{name}.maps.fld"), "r") as f:
            lines = f.readlines()
            for line in lines:
                if grid_name in line:
                    print(line)
                    print("Ligand in map fld")
                    # break
                    subprocess.run(autodock_command, check=True,cwd=flex_out)        
                    print("Docking successful")
                    docked.append((lig, pdb))
    except:
        pass
    # break
print(docked)

6T3D_SER_TDJ_flex
6T3D_SER_SR3_flex



autogrid4: ERROR:  unknown ligand atom type Ru
add parameters for it to the parameter library first!




#GRID_PARAMETER_FILE 6T3D_SER_SR3_flex.gpf

Ligand in map fld
AutoDock-GPU version: v1.5.3-53-gf58a5173fa2f4c82a74b40bc895ad7e65de59f47

Running 1 docking calculation

Cuda device:                              NVIDIA GeForce RTX 3060
Available memory on device:               11389 MB (total: 12022 MB)

CUDA Setup time 0.053499s

Running Job #1
    Using heuristics: (capped) number of evaluations set to 1132076
    Local-search chosen method is: ADADELTA (ad)

Rest of Setup time 0.002949s

Executing docking runs, stopping automatically after either reaching 0.15 kcal/mol standard deviation of
the best molecules of the last 4 * 5 generations, 42000 generations, or 1132076 evaluations:

Generations |  Evaluations |     Threshold    |  Average energy of best 10%  | Samples |    Best energy
------------+--------------+------------------+------------------------------+---------+-------------------
          0 |          150 |  451.14 kcal/mol |  329.52 +/-   32.43 kcal/mol |       4 |  287.3

prod_structures/Ligand_Substrate/PDBQT_Flex/APO_6T3D_ADT_rigid.maps.fld
6T3D_SER_3P7_flex.gpf
#GRID_PARAMETER_FILE 6T3D_SER_3P7_flex.gpf

Ligand in map fld


In [14]:
print(os.getcwd())

/home/alexi/Documents/SBL_MD_CovDock


In [21]:
import os

def get_directory_details(directory_path):
    total_size = 0
    file_count = 0
    
    for dirpath, dirnames, filenames in os.walk(directory_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):  # skip symbolic links
                total_size += os.path.getsize(fp)
            file_count += 1

    return file_count, total_size

directory_path = 'start_structures'
file_count, total_size = get_directory_details(directory_path)
print(f"Number of files: {file_count}")
print(f"Total size: {total_size} bytes")
print(f"Total size: {total_size / 1024**2} MB")

Number of files: 1484
Total size: 203830516 bytes
Total size: 194.3879280090332 MB
